In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [ ]:
# train_df = pd.read_csv("/content/gdrive/MyDrive/College/Semester5/ML/Project/twitter16/twitter16.csv")
# train_df = train_df[['source_tweet','label']].rename(columns = {'source_tweet':'text', 'label':'label'})

train_df = pd.read_csv("/content/gdrive/MyDrive/College/Semester5/ML/Project/kaggle/train.csv")
train_df = train_df.dropna()
train_df = train_df[['text','label']]

In [ ]:
trainDF = train_df
trainDF.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
trainDF.shape

(18285, 2)

In [ ]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
train_x.shape

(13713,)

In [ ]:
import re
from nltk.corpus import stopwords
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

def preprocess(raw_text):

    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)

    # convert to lower case and split 
    words = letters_only_text.lower().split()

    # remove stopwords
    stopword_set = set(stopwords.words("english"))
    meaningful_words = [w for w in words if w not in stopword_set]
    
    #stemmed words
    ps = PorterStemmer()
    stemmed_words = [ps.stem(word) for word in meaningful_words]
    
    #join the cleaned words in a list
    cleaned_word_list = " ".join(stemmed_words)

    return cleaned_word_list

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = trainDF
df['text'] = df['text'].apply(lambda line : preprocess(line))

In [ ]:
df.to_csv("/content/gdrive/MyDrive/College/Semester5/ML/Project/kaggle/processed.csv")

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def split_dataset_into_words(dataset):
    datawords = dataset.apply(lambda x: x.split())
    return list(datawords)

#  my_list = all_incidents 
# dictionnary
def buffer_stemmisation_keywords(my_list):
    my_list = [item for sublist in my_list for item in sublist]
    aux = pd.DataFrame(my_list, columns =['word'] )
    aux['word_stemmed'] = aux['word'].apply(lambda x : stemmer.stem(x))
    aux = aux.groupby('word_stemmed').transform(lambda x: ', '.join(x))
    aux['word_stemmed'] = aux['word'].apply(lambda x : stemmer.stem(x.split(',')[0]))
    aux.index = aux['word_stemmed']
    del aux['word_stemmed']
    my_dict = aux.to_dict('dict')['word']
    return my_dict

In [ ]:
dictionnary_all_words_unstemmed = buffer_stemmisation_keywords(split_dataset_into_words(df['text']))

# Dictionnary de-duplicated
for key, value in dictionnary_all_words_unstemmed.items():
    new_value = value.replace(",", "")
    new_value = list(set(value.split()))
    new_value = list(set(map(lambda each:each.strip(","), new_value)))
    dictionnary_all_words_unstemmed[key]=new_value

In [ ]:
df.to_csv("/content/gdrive/MyDrive/College/Semester5/ML/Project/kaggle/processed2.csv")

In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['text'], df['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=1900)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=1900)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=1900)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [ ]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('/content/gdrive/MyDrive/Amazon/productner/data/glove.6B.100d.txt')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y), metrics.precision_score(predictions, valid_y),metrics.recall_score(predictions, valid_y),metrics.f1_score(predictions, valid_y)

In [ ]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors

accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  (0.8932038834951457, 0.8444444444444444, 0.9047619047619048, 0.8735632183908046)
NB, WordLevel TF-IDF:  (0.9029126213592233, 0.8888888888888888, 0.8888888888888888, 0.8888888888888888)
NB, N-Gram Vectors:  (0.8737864077669902, 0.9333333333333333, 0.8076923076923077, 0.8659793814432989)
NB, CharLevel Vectors:  (0.9223300970873787, 0.9333333333333333, 0.8936170212765957, 0.9130434782608695)


In [ ]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)

LR, Count Vectors:  (0.8932038834951457, 0.8222222222222222, 0.925, 0.8705882352941177)
LR, WordLevel TF-IDF:  (0.9029126213592233, 0.8444444444444444, 0.926829268292683, 0.8837209302325582)
LR, N-Gram Vectors:  (0.8252427184466019, 0.8888888888888888, 0.7547169811320755, 0.8163265306122449)
LR, CharLevel Vectors:  (0.912621359223301, 0.8222222222222222, 0.9736842105263158, 0.891566265060241)


In [ ]:
# Linear Classifier on Count Vectors
accuracy = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)
print("SVM, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print("SVM, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("SVM, CharLevel Vectors: ", accuracy)

SVM, Count Vectors:  (0.912621359223301, 0.8444444444444444, 0.95, 0.8941176470588236)
SVM, WordLevel TF-IDF:  (0.912621359223301, 0.8444444444444444, 0.95, 0.8941176470588236)
SVM, N-Gram Vectors:  (0.8155339805825242, 0.9111111111111111, 0.7321428571428571, 0.8118811881188118)
SVM, CharLevel Vectors:  (0.9223300970873787, 0.8444444444444444, 0.9743589743589743, 0.9047619047619048)


In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
# Linear Classifier on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print("RF, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print("RF, WordLevel TF-IDF: ", accuracy)

# # Linear Classifier on Ngram Level TF IDF Vectors
# accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
# print("RF, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("RF, CharLevel Vectors: ", accuracy)

RF, Count Vectors:  (0.9029126213592233, 0.8, 0.972972972972973, 0.8780487804878049)
RF, WordLevel TF-IDF:  (0.883495145631068, 0.7777777777777778, 0.9459459459459459, 0.8536585365853658)
RF, CharLevel Vectors:  (0.9223300970873787, 0.8444444444444444, 0.9743589743589743, 0.9047619047619048)


In [ ]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print("Xgb, Count Vectors: ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print("Xgb, WordLevel TF-IDF: ", accuracy)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print("Xgb, CharLevel Vectors: ", accuracy)

Xgb, Count Vectors:  (0.8155339805825242, 0.7555555555555555, 0.8095238095238095, 0.7816091954022989)
Xgb, WordLevel TF-IDF:  (0.8446601941747572, 0.7555555555555555, 0.8717948717948718, 0.8095238095238095)
Xgb, CharLevel Vectors:  (0.9029126213592233, 0.8666666666666667, 0.9069767441860465, 0.8863636363636364)


In [ ]:
from tensorflow.keras import optimizers
import numpy as np

NN Baselines

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):

    classifier.fit(feature_vector_train, label, epochs=10)

    predictions = classifier.predict(feature_vector_valid)
    pred =[]
    for i in predictions:
        pred.append([np.round(i[0])])
    predictions = pred
    return metrics.accuracy_score(predictions, valid_y), metrics.precision_score(predictions, valid_y),metrics.recall_score(predictions, valid_y),metrics.f1_score(predictions, valid_y)

In [ ]:
def create_cnn():

    input_layer = layers.Input((70, ))

    embedding_layer = layers.Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], trainable = False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    pooling_layer = layers.GlobalAveragePooling1D()(conv_layer)

    output_layer2 = layers.Dense(1, activation="sigmoid")(pooling_layer)

    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)

print ("accuracy:-", accuracy[0])
print ("precision:-", accuracy[1])
print ("recall:-", accuracy[2])
print ("f1 score:-", accuracy[3])


Epoch 1/10
10/10 [==============================] - 1s 9ms/step - loss: 0.6836
Epoch 2/10
10/10 [==============================] - 0s 10ms/step - loss: 0.6688
Epoch 3/10
10/10 [==============================] - 0s 10ms/step - loss: 0.6520
Epoch 4/10
10/10 [==============================] - 0s 10ms/step - loss: 0.6338
Epoch 5/10
10/10 [==============================] - 0s 10ms/step - loss: 0.6117
Epoch 6/10
10/10 [==============================] - 0s 10ms/step - loss: 0.5858
Epoch 7/10
10/10 [==============================] - 0s 10ms/step - loss: 0.5659
Epoch 8/10
10/10 [==============================] - 0s 12ms/step - loss: 0.5403
Epoch 9/10
10/10 [==============================] - 0s 10ms/step - loss: 0.5124
Epoch 10/10
10/10 [==============================] - 0s 10ms/step - loss: 0.4895
accuracy:- 0.7864077669902912
precision:- 0.7111111111111111
recall:- 0.7804878048780488
f1 score:- 0.7441860465116279


In [ ]:
def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], trainable=True)(input_layer)
    # embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("RNN-LSTM, Word Embeddings",  accuracy)

Epoch 1/10
10/10 [==============================] - 3s 61ms/step - loss: 0.6711
Epoch 2/10
10/10 [==============================] - 1s 63ms/step - loss: 0.5727
Epoch 3/10
10/10 [==============================] - 1s 63ms/step - loss: 0.3600
Epoch 4/10
10/10 [==============================] - 1s 61ms/step - loss: 0.2347
Epoch 5/10
10/10 [==============================] - 1s 62ms/step - loss: 0.1301
Epoch 6/10
10/10 [==============================] - 1s 62ms/step - loss: 0.0909
Epoch 7/10
10/10 [==============================] - 1s 62ms/step - loss: 0.0418
Epoch 8/10
10/10 [==============================] - 1s 63ms/step - loss: 0.0286
Epoch 9/10
10/10 [==============================] - 1s 61ms/step - loss: 0.0162
Epoch 10/10
10/10 [==============================] - 1s 63ms/step - loss: 0.0072
RNN-LSTM, Word Embeddings (0.883495145631068, 0.8222222222222222, 0.9024390243902439, 0.8604651162790697)


In [ ]:
def create_rnn_gru():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], trainable=True)(input_layer)
    # embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_gru()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("RNN-GRU, Word Embeddings",  accuracy)

Epoch 1/10
10/10 [==============================] - 3s 58ms/step - loss: 0.6824
Epoch 2/10
10/10 [==============================] - 1s 58ms/step - loss: 0.6382
Epoch 3/10
10/10 [==============================] - 1s 59ms/step - loss: 0.5449
Epoch 4/10
10/10 [==============================] - 1s 57ms/step - loss: 0.3823
Epoch 5/10
10/10 [==============================] - 1s 56ms/step - loss: 0.2096
Epoch 6/10
10/10 [==============================] - 1s 57ms/step - loss: 0.1032
Epoch 7/10
10/10 [==============================] - 1s 56ms/step - loss: 0.0557
Epoch 8/10
10/10 [==============================] - 1s 57ms/step - loss: 0.0388
Epoch 9/10
10/10 [==============================] - 1s 56ms/step - loss: 0.0200
Epoch 10/10
10/10 [==============================] - 1s 55ms/step - loss: 0.0131
RNN-GRU, Word Embeddings (0.8737864077669902, 0.8666666666666667, 0.8478260869565217, 0.8571428571428571)


In [ ]:
def create_bidirectional_rnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], trainable=True)(input_layer)
    # embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_bidirectional_rnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("RNN-Bidirectional, Word Embeddings",  accuracy)

Epoch 1/10
10/10 [==============================] - 5s 102ms/step - loss: 0.6919
Epoch 2/10
10/10 [==============================] - 1s 100ms/step - loss: 0.6364
Epoch 3/10
10/10 [==============================] - 1s 99ms/step - loss: 0.5825
Epoch 4/10
10/10 [==============================] - 1s 101ms/step - loss: 0.3950
Epoch 5/10
10/10 [==============================] - 1s 98ms/step - loss: 0.2372
Epoch 6/10
10/10 [==============================] - 1s 99ms/step - loss: 0.1359
Epoch 7/10
10/10 [==============================] - 1s 98ms/step - loss: 0.0768
Epoch 8/10
10/10 [==============================] - 1s 99ms/step - loss: 0.0342
Epoch 9/10
10/10 [==============================] - 1s 98ms/step - loss: 0.0125
Epoch 10/10
10/10 [==============================] - 1s 99ms/step - loss: 0.0039
RNN-Bidirectional, Word Embeddings (0.8932038834951457, 0.8444444444444444, 0.9047619047619048, 0.8735632183908046)


In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, Lambda, Concatenate, Conv1D, GlobalMaxPooling1D


class RCNN(Model):
    def __init__(self,
                 maxlen,
                 max_features,
                 embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        super(RCNN, self).__init__()
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation
        self.embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen, trainable = False)
        self.forward_rnn = SimpleRNN(128, return_sequences=True)
        self.backward_rnn = SimpleRNN(128, return_sequences=True, go_backwards=True)
        self.reverse = Lambda(lambda x: tf.reverse(x, axis=[1]))
        self.concatenate = Concatenate(axis=2)
        self.conv = Conv1D(64, kernel_size=3, activation='relu')
        self.max_pooling = GlobalMaxPooling1D()
        self.classifier = Dense(self.class_num, activation=self.last_activation)

    def call(self, inputs):
        if len(inputs) != 3:
            raise ValueError('The length of inputs of RCNN must be 3, but now is %d' % len(inputs))
        input_current = inputs[0]
        input_left = inputs[1]
        input_right = inputs[2]
        if len(input_current.get_shape()) != 2 or len(input_left.get_shape()) != 2 or len(input_right.get_shape()) != 2:
            raise ValueError('The rank of inputs of RCNN must be (2, 2, 2), but now is (%d, %d, %d)' % (len(input_current.get_shape()), len(input_left.get_shape()), len(input_right.get_shape())))
        if input_current.get_shape()[1] != self.maxlen or input_left.get_shape()[1] != self.maxlen or input_right.get_shape()[1] != self.maxlen:
            raise ValueError('The maxlen of inputs of RCNN must be (%d, %d, %d), but now is (%d, %d, %d)' % (self.maxlen, self.maxlen, self.maxlen, input_current.get_shape()[1], input_left.get_shape()[1], input_right.get_shape()[1]))
        embedding_current = self.embedding(input_current)
        embedding_left = self.embedding(input_left)
        embedding_right = self.embedding(input_right)
        x_left = self.forward_rnn(embedding_left)
        x_right = self.backward_rnn(embedding_right)
        x_right = self.reverse(x_right)
        x = self.concatenate([x_left, embedding_current, x_right])
        x = self.conv(x)
        x = self.max_pooling(x)
        output = self.classifier(x)
        return output

In [ ]:

   
from tensorflow.keras import Model
from tensorflow.keras.layers import Embedding, Dense, Concatenate, Conv1D, Bidirectional, LSTM, GlobalAveragePooling1D, GlobalMaxPooling1D


class RCNNVariant(Model):
    """Variant of RCNN.
        Base on structure of RCNN, we do some improvement:
        1. Ignore the shift for left/right context.
        2. Use Bidirectional LSTM/GRU to encode context.
        3. Use Multi-CNN to represent the semantic vectors.
        4. Use ReLU instead of Tanh.
        5. Use both AveragePooling and MaxPooling.
    """

    def __init__(self,
                 maxlen,
                 max_features,
                 embedding_dims,
                 kernel_sizes=[1, 2, 3, 4, 5],
                 class_num=1,
                 last_activation='sigmoid'):
        super(RCNNVariant, self).__init__()
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.kernel_sizes = kernel_sizes
        self.class_num = class_num
        self.last_activation = last_activation
        self.embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)
        self.bi_rnn = Bidirectional(LSTM(128, return_sequences=True))
        self.concatenate = Concatenate()
        self.convs = []
        for kernel_size in self.kernel_sizes:
            conv = Conv1D(128, kernel_size, activation='relu')
            self.convs.append(conv)
        self.avg_pooling = GlobalAveragePooling1D()
        self.max_pooling = GlobalMaxPooling1D()
        self.classifier = Dense(self.class_num, activation=self.last_activation)

    def call(self, inputs):
        if len(inputs.get_shape()) != 2:
            raise ValueError('The rank of inputs of TextRNN must be 2, but now is %d' % len(inputs.get_shape()))
        if inputs.get_shape()[1] != self.maxlen:
            raise ValueError('The maxlen of inputs of TextRNN must be %d, but now is %d' % (self.maxlen, inputs.get_shape()[1]))
        embedding = self.embedding(inputs)
        x_context = self.bi_rnn(embedding)
        x = self.concatenate([embedding, x_context])
        convs = []
        for i in range(len(self.kernel_sizes)):
            conv = self.convs[i](x)
            convs.append(conv)
        poolings = [self.avg_pooling(conv) for conv in convs] + [self.max_pooling(conv) for conv in convs]
        x = self.concatenate(poolings)
        output = self.classifier(x)
        return output

In [ ]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

max_features = 5000
maxlen = 40
batch_size = 32
embedding_dims = 50


print('Loading data...')
df = pd.read_csv("/content/gdrive/MyDrive/College/Semester5/ML/Project/twitter15/twitter15.csv")
df = df.rename(columns = {'source_tweet':'message', 'label':'class'})
df = df.sample(frac=1)
train = df[:int(0.80*df.shape[0])]
test = df[int(0.80*df.shape[0]):]
print(train.columns)
print(test.columns)
x_train, y_train, x_test, y_test = train['message'], train['class'], test['message'], test['class']
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)...')
# x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
# x_test = sequence.pad_sequences(x_test, maxlen=maxlen)


embeddings_index = {}
for i, line in enumerate(open('/content/gdrive/MyDrive/Amazon/productner/data/glove.6B.100d.txt')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

token = text.Tokenizer()
token.fit_on_texts(train['message'])
word_index = token.word_index

x_train = sequence.pad_sequences(token.texts_to_sequences(x_train), maxlen=maxlen)
x_test = sequence.pad_sequences(token.texts_to_sequences(x_test), maxlen=maxlen)


# print('Prepare input for model...')
# x_train_current = x_train
# x_train_left = np.hstack([np.expand_dims(x_train[:, 0], axis=1), x_train[:, 0:-1]])
# x_train_right = np.hstack([x_train[:, 1:], np.expand_dims(x_train[:, -1], axis=1)])
# x_test_current = x_test
# x_test_left = np.hstack([np.expand_dims(x_test[:, 0], axis=1), x_test[:, 0:-1]])
# x_test_right = np.hstack([x_test[:, 1:], np.expand_dims(x_test[:, -1], axis=1)])
# print('x_train_current shape:', x_train_current.shape)
# print('x_train_left shape:', x_train_left.shape)
# print('x_train_right shape:', x_train_right.shape)
# print('x_test_current shape:', x_test_current.shape)
# print('x_test_left shape:', x_test_left.shape)
# print('x_test_right shape:', x_test_right.shape)


Loading data...
Index(['tweetID', 'message', 'class', 'tree'], dtype='object')
Index(['tweetID', 'message', 'class', 'tree'], dtype='object')
592 train sequences
149 test sequences
Pad sequences (samples x time)...


In [ ]:
print('Prepare input for model...')
x_train_current = x_train
x_train_left = np.hstack([np.expand_dims(x_train[:, 0], axis=1), x_train[:, 0:-1]])
x_train_right = np.hstack([x_train[:, 1:], np.expand_dims(x_train[:, -1], axis=1)])
x_test_current = x_test
x_test_left = np.hstack([np.expand_dims(x_test[:, 0], axis=1), x_test[:, 0:-1]])
x_test_right = np.hstack([x_test[:, 1:], np.expand_dims(x_test[:, -1], axis=1)])
print('x_train_current shape:', x_train_current.shape)
print('x_train_left shape:', x_train_left.shape)
print('x_train_right shape:', x_train_right.shape)
print('x_test_current shape:', x_test_current.shape)
print('x_test_left shape:', x_test_left.shape)
print('x_test_right shape:', x_test_right.shape)

Prepare input for model...
x_train_current shape: (592, 40)
x_train_left shape: (592, 40)
x_train_right shape: (592, 40)
x_test_current shape: (149, 40)
x_test_left shape: (149, 40)
x_test_right shape: (149, 40)


In [ ]:
epochs = 10
print('Build model...')
model = RCNN(maxlen, max_features, embedding_dims)
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, mode='max')
model.fit([x_train_current, x_train_left, x_train_right], y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=([x_test_current, x_test_left, x_test_right], y_test))

print('Test...')
result = model.predict([x_test_current, x_test_left, x_test_right])



# print('Build model...')
# model = RCNNVariant(maxlen, max_features, embedding_dims)
# model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

# print('Train...')
# early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, mode='max')
# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           callbacks=[early_stopping],
#           validation_data=(x_test, y_test))

# print('Test...')
# result = model.predict(x_test)

Build model...
Train...
Epoch 1/10
19/19 [==============================] - 3s 60ms/step - loss: 0.6965 - accuracy: 0.5034 - val_loss: 0.7022 - val_accuracy: 0.4564
Epoch 2/10
19/19 [==============================] - 1s 40ms/step - loss: 0.6438 - accuracy: 0.6047 - val_loss: 1.1105 - val_accuracy: 0.4564
Epoch 3/10
19/19 [==============================] - 1s 39ms/step - loss: 0.6440 - accuracy: 0.6537 - val_loss: 0.7201 - val_accuracy: 0.5906
Epoch 4/10
19/19 [==============================] - 1s 40ms/step - loss: 0.6026 - accuracy: 0.6740 - val_loss: 0.6809 - val_accuracy: 0.5369
Epoch 5/10
19/19 [==============================] - 1s 40ms/step - loss: 0.5635 - accuracy: 0.6892 - val_loss: 0.6417 - val_accuracy: 0.6711
Epoch 6/10
19/19 [==============================] - 1s 40ms/step - loss: 0.5010 - accuracy: 0.7669 - val_loss: 0.7032 - val_accuracy: 0.6309
Epoch 7/10
19/19 [==============================] - 1s 40ms/step - loss: 0.4444 - accuracy: 0.8209 - val_loss: 0.6695 - val_accura

In [ ]:
result = (result.reshape(1,-1)[0]>0.5).astype(int)
result

array([1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1])

In [ ]:
print("ACC", metrics.accuracy_score(result, y_test))
print("Prec", metrics.precision_score(result, y_test))
print("REC", metrics.recall_score(result, y_test))
print("F1", metrics.f1_score(result, y_test))

ACC 0.6375838926174496
Prec 0.7941176470588235
REC 0.574468085106383
F1 0.6666666666666666


Explaination

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline, make_pipeline

vec = TfidfVectorizer(min_df=3, stop_words='english',
                      ngram_range=(1, 2))

# The dimension of the input documents is reduced to 100, and then a kernel SVM is used to classify the documents.
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
lsa = make_pipeline(vec, svd)

clf = SVC(C=150, gamma=2e-2, probability=True)
pipe = make_pipeline(lsa, clf)
pipe.fit(train_x.to_numpy(), train_y)
pipe.score(valid_x.to_numpy(), valid_y)

0.883495145631068

In [ ]:
valid_x, train_x = valid_x.to_numpy(), train_x.to_numpy()

In [ ]:
def print_prediction(doc):
    y_pred = pipe.predict_proba([doc])[0]
    for target, prob in zip(np.unique(train_y), y_pred):
        print("{:.3f} {}".format(prob, target))

doc = valid_x[0]

print(valid_x[0])
print('------------------------------------ What is the prediction?-------------------------------------------------------')
print_prediction(doc)

cop arrest dylann roof treat burger king url url
------------------------------------ What is the prediction?-------------------------------------------------------
0.978 0
0.022 1


### TextExplainer

In [ ]:
pip install eli5

     |████████████████████████████████| 106 kB 6.9 MB/s 


In [ ]:
import eli5
from eli5.lime import TextExplainer
doc = train_x[2]
te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction(target_names=['fake', 'real'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
doc = train_x[0]
te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction(target_names=['fake', 'real'])

In [ ]:
doc = train_x[1]
te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction(target_names=['fake', 'real'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
doc = train_x[2]
te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction(target_names=['fake', 'real'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
train_x[2]

'exclus michael zahaf bibeau caught dashcam model sold url ottawashoot url'

In [ ]:
doc = train_x[3]
te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction(target_names=['fake', 'real'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
doc = train_x[4]
te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction(target_names=['fake', 'real'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
doc = train_x[5]
te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction(target_names=['fake', 'real'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
doc = train_x[7]
te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction(target_names=['fake', 'real'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
